In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import math as time
import mysql.connector 
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
import pymysql
from sqlalchemy import create_engine

In [33]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="test"
)

In [34]:
cbs = pd.read_sql("Select * from htc",con=mydb)
#tblpatientspriorexposure

In [35]:
cbs.tail()

,HTCNumber,OPDNumber,DateofRegistration,CoupleTo,isCouple,HTCApproach,HTCModel_Id,CounselingRecieved,OptedOut,NameofCounselor,...,RefferedforService,TestPurposeType_ID,ReferredFromType_ID,tested_before,current_pregnancy_lactation,date_tested_before,EntryPointID,FinalResultType,PerformedBy,notified
28,06-06-25-2019-V-00475,6-6-25-2019-09-19-08-42-02.979,2019-08-22,,No,None,6,Yes,No,SIFISO NKOMO,...,",3,4,5,9,10",2,NaN,No,,NaT,1,Positive,sifiso nkomo,0
29,06-06-25-2019-V-00500,6-6-25-2019-644,2019-01-09,,No,PITC,6,Individual Pre-Test Information,No,SITHULISIWE NHLANE,...,",3,4,5,7,10",15,6.0,No,,NaT,1,Positive,SITHULISIWE NHLANE,0
30,06-06-25-2019-V-00505,6-6-25-2019-662,2019-04-29,,No,PITC,6,Individual Pre-Test Information,No,SIFISO NKOMO,...,",3,4,9,10",13,99.0,No,,NaT,3,Positive,SIFISO NKOMO,0
31,06-06-25-2019-V-00514,6-6-25-2019-665,2019-06-24,,No,PITC,6,Individual Pre-Test Information,No,SIFISO NKOMO,...,",3,4,5,9,10",13,99.0,Yes,No,NaT,3,Positive,SIFISO NKOMO,2
32,06-06-25-2019-V-00524,6-6-25-2019-09-19-08-59-08.539,2019-09-12,,No,None,6,Yes,No,SIFISO NKOMO,...,",3,4,5,9,10",13,NaN,Yes,No,2018-08-01,10,Positive,SIFISO NKOMO,2


In [36]:
cbs = pd.read_sql("Select * from tblvisits",con=mydb)

In [37]:
cbs.tail()

,PatientID,VisitDate,VisitType,VisitTypeCode,NowPregnant,FamilyPlanningID,FunctionalStatusCode,WHOStage,TBStatusCode,IPTReasonCode,...,StaffID,UserNumber,TheTimeStamp,AdverseEventsStatusCode,DispenserID,DateOfFirstANC,LNMPDate,TBInvestigationResults,IsIPTEligibility,IPTStatusCode
10779,09-0D-02-2014-A-00167,2018-08-17,None,P,None,O,W,3.0,Y,None,...,3.0,76,2018-08-23 14:45:54,None,NaN,None,None,NaN,None,None
10780,09-0D-02-2014-A-00167,2018-11-19,None,P,None,O,W,3.0,Y,None,...,1.0,78,2018-11-21 10:43:13,None,NaN,None,None,NaN,None,None
10781,09-0D-02-2014-A-00167,2019-02-11,None,TS,None,None,None,NaN,None,None,...,3.0,78,2019-05-15 11:48:58,None,NaN,None,None,NaN,None,None
10782,09-0D-02-2014-A-00167,2019-05-07,None,TS,None,None,None,NaN,None,None,...,3.0,78,2019-05-15 11:50:33,None,NaN,None,None,NaN,None,None
10783,09-0D-02-2014-A-00167,2019-07-31,CL,P,N/A,None,W,3.0,NO,None,...,NaN,3,NaT,,0.0,None,None,0.0,,


In [38]:
statcode = pd.read_sql("Select * from tblsetuparvstatuscodes",con=mydb)

In [39]:
statcode.head()

,ARVStatusCode,ARVStatusDescription,OnARVs
0,1,No ARV,0
1,2,Start ARV,-1
2,3,Continue ARV,-1
3,4,Change ARV,-1
4,5,Stop ARV,0


In [40]:
statcode = pd.read_sql("Select * from tblsetuparvstatusreasonmap",con=mydb)

In [41]:
statcode.head()

,ARVReasonCode,ARVStatusCode
0,11,1
1,12,1
2,13,1
3,14,1
4,15,1


In [42]:
cbs1 = pd.read_sql("Select patient.patientid, cbs.HTCNumber,cbs.DateFormCompleted,cbs.WhereTestDone,cbs.dateOfBirth,cbs.sex,cbs.religion,cbs.employmentStatus,cbs.maritalStatus,cbs.districtOfResidence, patient.RegistrationDate,htc.FinalResultType,htc.tested_before,htc.date_tested_before,tblsetuptesttypes.TestType,tbltests.ResultNumeric,tbltests.TestDate,tblvisits.VisitDate,tblvisits.WHOStage,tblsetuparvstatuscodes.ARVStatusDescription,tblstatus.Status FROM cbs JOIN patient ON cbs.OPDNumber = patient.OPDNumber JOIN htc ON cbs.HTCNumber = htc.HTCNumber JOIN tbltests ON patient.PatientID = tbltests.PatientID JOIN tblvisits ON patient.PatientID = tblvisits.PatientID JOIN tblstatus ON  patient.PatientID = tblstatus.PatientID JOIN tblsetuptesttypes ON tbltests.TestTypeID = tblsetuptesttypes.TestTypeID JOIN tblsetuparvstatuscodes ON tblvisits.ARVStatusCode = tblsetuparvstatuscodes.ARVStatusCode",con=mydb)

In [43]:
cbs1.tail()

,patientid,HTCNumber,DateFormCompleted,WhereTestDone,dateOfBirth,sex,religion,employmentStatus,maritalStatus,districtOfResidence,...,FinalResultType,tested_before,date_tested_before,TestType,ResultNumeric,TestDate,VisitDate,WHOStage,ARVStatusDescription,Status
17746,06-06-25-2019-A-00013,06-06-25-2019-V-00141,2019-03-04,None,1996-06-15,Female,other specify,unemployed,married,UMZINGWANE,...,Positive,Yes,NaT,CD4 count,497.0,2019-05-25,2019-07-23,3.0,Change ARV,Attending this clinic
17747,06-06-25-2019-A-00013,06-06-25-2019-V-00141,2019-03-04,None,1996-06-15,Female,other specify,unemployed,married,UMZINGWANE,...,Positive,Yes,NaT,Weight,46.0,2019-06-25,2019-07-23,3.0,Change ARV,Attending this clinic
17748,06-06-25-2019-A-00013,06-06-25-2019-V-00141,2019-03-04,None,1996-06-15,Female,other specify,unemployed,married,UMZINGWANE,...,Positive,Yes,NaT,Weight,46.0,2019-06-25,2019-07-23,3.0,Change ARV,Attending this clinic
17749,06-06-25-2019-A-00013,06-06-25-2019-V-00141,2019-03-04,None,1996-06-15,Female,other specify,unemployed,married,UMZINGWANE,...,Positive,Yes,NaT,Weight,46.0,2019-06-25,2019-07-23,3.0,Change ARV,Attending this clinic
17750,06-06-25-2019-A-00013,06-06-25-2019-V-00141,2019-03-04,None,1996-06-15,Female,other specify,unemployed,married,UMZINGWANE,...,Positive,Yes,NaT,Weight,46.0,2019-06-25,2019-07-23,3.0,Change ARV,Attending this clinic


In [44]:
filename = cbs1.to_csv (r'/media/devoop/GINA/ePMS backups/CBS/esibomvu.csv', index = None, header=True)

In [1]:
mydb.close()

NameError: name 'mydb' is not defined